In [ ]:
import pandas as pd
df=pd.read_csv('kc_house_data.csv')
df['date'] = pd.to_datetime(df.date)
df['bedrooms'] = df[['bedrooms','sqft_living']].apply(lambda x: 3 if (x.sqft_living > 1000)&(x.bedrooms==0) else x.bedrooms, axis=1)
from scipy.stats import boxcox
import numpy as np
df['log_living']= np.log(df['sqft_living'])
df.log_living.skew()
df['log_lot']= np.log(df['sqft_lot'])
df.log_lot.skew()
from scipy.stats import boxcox
df['boxcox_lot']= boxcox(df.sqft_lot)[0]
df.boxcox_lot.skew()
df['lot/inside']= df['sqft_lot']/df['sqft_living']
df['base/upper']= df['sqft_basement']/df['sqft_living']
df['sqft_liv/15']= df['sqft_living']/df['sqft_living15']
df['sqft_LOT/15']= df['sqft_lot']/df['sqft_lot15']
from datetime import date
today = date.today() 
df['remodel_age']= df['yr_renovated'].apply(lambda x: today.year - x if (x > 0) else 0)
from scipy import stats
linear_regression=stats.linregress(df['sqft_living'], df['price'])
linear_regression
def price_call(sqft_living):
    price=sqft_living * linear_regression.slope + linear_regression.intercept
    return price
df['predicted_price'] = price_call(df['sqft_living'])
import numpy as py
from sklearn.linear_model import LinearRegression
l = LinearRegression()
x = df[['remodel_age','zipcode','log_living', 'grade', 'sqft_above','boxcox_lot', 'sqft_living15', 'bathrooms', 'view', 'sqft_basement','bedrooms', 'sqft_liv/15']]

y = df[['price']]
x2 = pd.merge(x, zipcode, left_index=True, right_index=True)
x2.drop('zipcode', axis=1, inplace=True)
zipcode = pd.get_dummies(x.zipcode)
l.fit(x2,y)
df['predict_price'] = l.predict(x2)
df['diff2']= df.price-df.predict_price
df['square_diff2'] = py.square(df['diff2'])
sum2=sum(df.square_diff2)
sum2
py.sqrt(sum2/len(df.square_diff2))
from sklearn.metrics import mean_squared_error
pred = l.predict(x2)
py.sqrt(mean_squared_error(df['price'], pred))
